In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp_hyper = Experiment(workspace=ws, name="udacity-project-hyperdrive")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp_hyper.start_logging()

Workspace name: quick-starts-ws-244938
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-244938


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "udacity-project1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Check if the compute cluster already exists, if not, create a new one
if cluster_name not in ws.compute_targets:
    # Configure the provisioning details for the compute cluster
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)

    # Create the compute cluster
    compute_target = ComputeTarget.create(ws, cluster_name, provisioning_config)

    # Wait for the compute cluster to be provisioned
    compute_target.wait_for_completion(show_output=True)
else:
    compute_target = ws.compute_targets[cluster_name]


InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
#ps = ### YOUR CODE HERE ###
# Define the hyperparameter search space  
ps = RandomParameterSampling({  
    '--C': choice(0.01, 0.1, 1.0, 10.0),  
    '--max_iter': choice(50, 100, 150, 200)  
})

# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)  

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#src = ### YOUR CODE HERE ###
src = ScriptRunConfig(source_directory='.',  
                                   script='train.py',  
                                   compute_target=compute_target,
                                   environment=sklearn_env) 

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,  
                                    hyperparameter_sampling=ps,  
                                    primary_metric_name='accuracy',  
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,  
                                    max_total_runs=20,  
                                    max_concurrent_runs=4)  

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

# Create an experiment  
# experiment = Experiment(ws, 'hyperdrive_experiment')  
  
# Submit the HyperDrive run to the experiment  
hyperdrive_run = exp_hyper.submit(config=hyperdrive_config)  
  
# Use the RunDetails widget to visualize the run details  
RunDetails(hyperdrive_run).show()  

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [16]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

# Get the best run from the HyperDrive experiment
best_run = hyperdrive_run.get_best_run_by_primary_metric()

print(best_run)


None


In [18]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

# Create a TabularDataset from a URL
url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

# Create a TabularDatasetFactory object
factory = TabularDatasetFactory()

# Create a TabularDataset using the TabularDatasetFactory
tabular_dataset_aml = factory.from_delimited_files(url)
  


In [19]:
from train import clean_data

# Use the clean_data function to clean your data.
# x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(tabular_dataset_aml)

In [27]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=tabular_dataset_aml,
    label_column_name='y',
    n_cross_validations=5)

In [28]:
# Submit your automl run

exp_aml = Experiment(workspace=ws, name='udacity-project-automl')  

### YOUR CODE HERE ###
automl_run = exp_aml.submit(automl_config)


2023-12-11:20:07:33,800 WARNING  [local_experiment_driver.py:142] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-automl,AutoML_efc18593-b0fb-4c3b-973f-d9659e434e98,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2023/12/11 20:16:15 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023-12-11:20:40:59,105 INFO     [explanation_client.py:334] Using default datastore for uploads


In [30]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = automl_run.get_output()
print(best_run)

print(fitted_model)
# model = automl_run.register_model(model_name='best_model_automl', model_path='./outputs/model_aml.pkl')

# best_model_path = './outputs/model_aml.pkl'
# fitted_model.save(best_model_path)


Run(Experiment: udacity-project-automl,
Id: AutoML_efc18593-b0fb-4c3b-973f-d9659e434e98_22,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('21', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.5, eta=0.2, gamma=0, max_depth=7, max_leaves=7, n_estimators=25, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'})

In [ ]:
# Get the AmlCompute cluster by name
compute_cluster = AmlCompute(ws, "udacity-project1")

# Delete the AmlCompute cluster
compute_cluster.delete()
